# Preprocesamiento

In [201]:
import pandas as pd

import sys
sys.path.append("..")

from src.support_prep import *
from src.support_models import *    

import pickle

In [202]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [203]:
df = pd.read_pickle("../datos/clean.pkl")

In [204]:
df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeID,Gender,JobLevel,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,51.0,No,Travel_Rarely,Sales,6.0,2.0,Life Sciences,1.0,Female,1.0,...,1.0,6.0,1.0,0.0,0.0,3.0,4.0,2.0,3.0,3.0
1,31.0,Yes,Travel_Frequently,Research & Development,10.0,1.0,Life Sciences,2.0,Female,1.0,...,6.0,3.0,5.0,1.0,4.0,3.0,2.0,4.0,2.0,4.0
2,32.0,No,Travel_Frequently,Research & Development,17.0,4.0,Other,3.0,Male,4.0,...,5.0,2.0,5.0,0.0,3.0,2.0,2.0,1.0,3.0,3.0
3,38.0,No,Non-Travel,Research & Development,2.0,5.0,Life Sciences,4.0,Male,3.0,...,13.0,5.0,8.0,7.0,5.0,4.0,4.0,3.0,2.0,3.0
4,32.0,No,Travel_Rarely,Research & Development,10.0,1.0,Medical,5.0,Male,1.0,...,9.0,2.0,6.0,0.0,4.0,4.0,1.0,3.0,3.0,3.0


## Encoding

Para realizar el encoding, vamos a necesitar convertir nuestra variable respuesta en binario.

In [205]:
df["Attrition"] = df["Attrition"].map({"No":0, "Yes":1})

In [206]:
cat_cols = df.select_dtypes("O").columns
num_cols = df.select_dtypes("number").columns.drop(["Attrition", "EmployeeID"])

In [207]:
df = df.drop(columns="EmployeeID")

In [208]:
cat_diff = detectar_orden_cat(df = df, lista_cat=cat_cols,var_respuesta="Attrition", show=False)

In [209]:
cat_no_diff = cat_cols.drop(cat_diff)

In [210]:
df_target, target = encode_target(data = df, columns = cat_diff, response_var="Attrition")

In [211]:
df_oh, onehot = encode_onehot(df, columns = cat_no_diff)

In [212]:
df = pd.concat([df["Attrition"], df_target.drop(columns=cat_no_diff), df_oh], axis = 1)

In [213]:
with open('../models/onehot.pkl', 'wb') as file:
    pickle.dump(onehot, file)
with open('../models/target.pkl', 'wb') as file:
    pickle.dump(target, file)

## Feature Scaling

In [214]:
df_scaled, scaler = scale_data(data = df, columns=df.columns.drop(df_oh.columns).drop("Attrition"), method = "minmax")

In [215]:
df[df_scaled.columns] = df_scaled

In [216]:
with open('../models/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## Outliers

In [219]:
df_outliers, ifo = find_outliers(data = df.drop(columns = "Attrition"), columns = df.drop(columns = "Attrition").columns, method = "ifo")

100%|██████████| 25/25 [01:17<00:00,  3.10s/it]


In [220]:
df.drop(index=df_outliers.index, inplace=True)

In [221]:
df.reset_index(drop=True, inplace=True)

In [222]:
df_outliers, lof = find_outliers(data = df.drop(columns = "Attrition"), columns = df.drop(columns = "Attrition").columns, method = "lof")

100%|██████████| 25/25 [00:02<00:00,  8.70it/s]


In [223]:
df.loc[df_outliers.index, num_cols] = np.nan

In [224]:
df_out_impute, out_imputer = impute_nulls(df)

[IterativeImputer] Completing matrix with shape (4318, 38)
[IterativeImputer] Ending imputation round 1/10, elapsed time 23.10
[IterativeImputer] Change: 1.1686352949733543, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Ending imputation round 2/10, elapsed time 47.12
[IterativeImputer] Change: 0.76634811444325, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Ending imputation round 3/10, elapsed time 72.38
[IterativeImputer] Change: 0.49739678077655425, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Ending imputation round 4/10, elapsed time 99.01
[IterativeImputer] Change: 0.583858914023746, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Ending imputation round 5/10, elapsed time 127.38
[IterativeImputer] Change: 0.49759133511224474, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Ending imputation round 6/10, elapsed time 157.22
[IterativeImputer] Change: 0.5037420126028298, scaled tolerance: 0.0010000000000000002 
[Ite

In [225]:
df = df_out_impute

In [226]:
with open('../models/ifo.pkl', 'wb') as file:
    pickle.dump(ifo, file)
with open('../models/lof.pkl', 'wb') as file:
    pickle.dump(lof, file)
with open('../models/out_imputer.pkl', 'wb') as file:
    pickle.dump(out_imputer, file)

In [227]:
df.to_pickle("../datos/prepped.pkl")